<a href="https://colab.research.google.com/github/AcidCannon/CMPUT466-Mini-project/blob/master/task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torchvision import datasets, transforms
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
from torchvision.utils import save_image

In [0]:
config = {}
config['training_size'] = 50000
config['training_batch_size'] = 64
config['training_shuffle'] = True
config['device'] = 'gpu'
config['generator_learning_rate'] = 0.0001
config['generator_weight_decay'] = 0.001
config['discriminator_learning_rate'] = 0.0001 
config['discriminator_weight_decay'] = 0.001
config['algorithm'] = 'Generative Adversarial Network'
config['number_of_epochs'] = 1000
config['discriminator_real_loss_coe'] = 1.0
config['discriminator_fake_loss_coe'] = 1.0

In [0]:
def load_data(config):
  MNIST_training_set = datasets.MNIST(root='data', train=True, download=True, transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.1307,), (0.3081,))
                  ]))
  
  MNIST_training_set = torch.utils.data.Subset(MNIST_training_set, range(0, config['training_size']))

  training_dataloader = torch.utils.data.DataLoader(dataset=MNIST_training_set, batch_size=config['training_batch_size'], shuffle=config['training_shuffle'])

  return training_dataloader

In [0]:
class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()
    self.fc1 = nn.Linear(10, 10)
    self.fc2 = nn.Linear(10, 64)
    self.fc3 = nn.Linear(64, 128)
    self.fc4 = nn.Linear(128, 784)
  
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    return torch.tanh(self.fc4(x))

In [0]:
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()
    self.fc1 = nn.Linear(784, 128)
    self.fc2 = nn.Linear(128, 64)
    self.fc3 = nn.Linear(64, 10)
    self.fc4 = nn.Linear(10, 1)
  
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    return torch.sigmoid(self.fc4(x))

In [0]:
def discriminator_train(config, data, discriminator_optimizer, discriminator, generator, loss_function):
  real_data = data.view(-1, 784)
  real_data = real_data.to(config['device'])
  real_target = torch.ones(data.shape[0], dtype=torch.long)
  real_target = real_target.to(config['device'])

  # set gradient to be zero
  discriminator_optimizer.zero_grad()

  # forward propagation for real
  output = discriminator(real_data)
  # compute loss for real
  discriminator_real_loss = loss_function(output, real_target)

  fake_data = torch.randn(data.shape[0], 10).to(config['device'])
  fake_data = generator(fake_data)
  fake_target = torch.zeros(data.shape[0], dtype=torch.long)
  fake_target = fake_target.to(config['device'])

  # forward propagation for fake
  output = discriminator(fake_data)
  # compute loss for fake
  discriminator_fake_loss = loss_function(output, fake_target)

  # compute total loss as a combination of real loss and fake loss
  print(discriminator_real_loss, discriminator_fake_loss)
  discriminator_total_loss = config['discriminator_real_loss_coe']*discriminator_real_loss + config['discriminator_fake_loss_coe']*discriminator_fake_loss
  
  # backward propagation for total loss
  discriminator_total_loss.backward()
  
  # weight updation
  discriminator_optimizer.step()
  
  return discriminator_total_loss

In [0]:
def generator_train(config, data, generator_optimizer, discriminator, generator, loss_function):
  # set gradient to be zero
  generator_optimizer.zero_grad()

  fake_data = torch.randn(data.shape[0], 10).to(config['device'])
  real_target = torch.ones(data.shape[0], dtype=torch.long)
  real_target = real_target.to(config['device'])

  # let generator generates fake image and see if discriminator can discriminate it
  # first forward propagate for generator
  # then forward propagate for discriminator
  output = discriminator(generator(fake_data))

  # compute loss for generator
  generator_total_loss = loss_function(output, real_target)

  # backward propagation for total loss
  generator_total_loss.backward()

  # weight updation
  generator_optimizer.step()

  return generator_total_loss

In [0]:
def train(training_dataloader, config, device):
  #### MAY MODIFY LATER ####
  loss_function = F.cross_entropy
  discriminator = Discriminator().to(device)
  discriminator.train()
  generator = Generator().to(device)
  generator.train()
  discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=config['discriminator_learning_rate'], weight_decay = config['discriminator_weight_decay'])
  generator_optimizer = torch.optim.Adam(generator.parameters(), lr=config['generator_learning_rate'], weight_decay=config['generator_weight_decay'])
  #### MAY MODIFY END ####
  for epoch in range(1, config['number_of_epochs']+1):
    discriminator_loss = 0
    generator_loss = 0
    for idx, (data, target) in enumerate(training_dataloader):
      discriminator_loss += discriminator_train(config, data, discriminator_optimizer, discriminator, generator, loss_function)
      generator_loss += generator_train(config, data, generator_optimizer, discriminator, generator, loss_function)
    
    print("Epoch: {}.\tGenerator Avg.loss: {}.\tDiscriminator Avg.loss: {}.".format(epoch, generator_loss, discriminator_loss))

In [280]:
if config['device'] != 'cpu' and torch.cuda.is_available():
  config['device'] = torch.device('cuda')
  print('Using GPU: {}.'.format(torch.cuda.get_device_name(0)))
else:
  config['device'] = torch.device('cpu')
  print('Using CPU.')
  
train(load_data(config), config, config['device'])

Using GPU: Tesla P100-PCIE-16GB.
tensor(0., device='cuda:0', grad_fn=<NllLossBackward>) tensor(0., device='cuda:0', grad_fn=<NllLossBackward>)
tensor(-0., device='cuda:0', grad_fn=<NllLossBackward>) tensor(0., device='cuda:0', grad_fn=<NllLossBackward>)
tensor(-0., device='cuda:0', grad_fn=<NllLossBackward>) tensor(0., device='cuda:0', grad_fn=<NllLossBackward>)
tensor(-0., device='cuda:0', grad_fn=<NllLossBackward>) tensor(0., device='cuda:0', grad_fn=<NllLossBackward>)
tensor(-0., device='cuda:0', grad_fn=<NllLossBackward>) tensor(0., device='cuda:0', grad_fn=<NllLossBackward>)
tensor(-0., device='cuda:0', grad_fn=<NllLossBackward>) tensor(0., device='cuda:0', grad_fn=<NllLossBackward>)
tensor(-0., device='cuda:0', grad_fn=<NllLossBackward>) tensor(0., device='cuda:0', grad_fn=<NllLossBackward>)
tensor(-0., device='cuda:0', grad_fn=<NllLossBackward>) tensor(0., device='cuda:0', grad_fn=<NllLossBackward>)
tensor(-0., device='cuda:0', grad_fn=<NllLossBackward>) tensor(0., device='cuda:

KeyboardInterrupt: ignored